# <b>機械学習の基礎-3</b>
アンサンブルや特徴量エンジニアリングについてメモしておく。  

参考：  
https://datawokagaku.com/ensemble/

## アンサンブル
複数の機械学習モデルを組み合わせて予測する手法。  
一般的に単一のモデルよりも精度が高く、実務での予測はアンサンブルが主流。  
アンサンブルは主にバギング、ブースティング、スタッキングの3種類がある。

組み合わせるモデルはそれぞれの相関が低い（似ていない）モデルが望ましく、できるだけ多種多様なモデルを用いる。  
※弱学習器を使うのが良いようだが、KaggleではGBDTとニューラルネットの組み合わせを行っている例もあるので、  
　必ずというわけではなさそう。

### バギング(bagging: bootstrap aggregating)
下記の流れで予測するのがバギング。
1. 学習データを母集団として、重複を許してランダムに標本抽出する。    
    （重複を許して： 同じレコードが何回出てきてもよい。）
2. 抽出した学習データでモデルに学習させる。
3. 学習させたいモデルの数だけ1.～2.を繰り返す（並列にモデルを複数個作るイメージ）。
4. 全学習済みモデルの予測結果の多数決や確率の平均をとり、それをバギングによる予測結果とする。  

学習データの一部しか使わずに学習させるため、high bias low variance気味になるので、  
モデルには高varianceになりやすい決定木を用いることが多い。

### ランダムフォレスト(Random Forest)
バギングにおいて学習モデルに決定木を採用し、さらに特徴量を選択して学習させるアンサンブル手法。  
それぞれの学習モデルで一部の学習データと特徴量を使わないことで、少し異なる決定木を複数作る。  
（相関が低いモデルを作ることが狙い。アンサンブルでは多種多様なモデルを用いることが望ましいため。）

### ブースティング(boosting)
バギングと異なり、直列に弱学習器を順次作っていき、  
それらの予測値ごとに重みをつけて足し合わせたものを予測値とする手法。  
下記の流れのように、「残差を目的変数として学習し、その予測値で残差を埋める」というプロセスを繰り返して、  
それぞれのモデルでの残差の予測値を重みをつけて足し合わせた結果を最終的な予測値とする。  

＜ブースティングの大まかな流れ（アルゴリズムによって異なる）＞  
ブースティングでの最終的な予測結果を$\hat{f}(x)$、  
残差を$r$（$i$番目のデータの残差は$r_i$）とおく。  
また、最初のイテレーションでは$\hat{f}(x)=0$とする。  
このとき、予測値が常に0なので残差は目的変数に等しい（$r_i=y_i$）。

以下の手順をN回繰り返す。(N = 1,2,...,N)
1. 弱学習器$\hat{f_t}$を学習データ$(X,r)$で学習
2. $r_i$を $r_i - \eta\hat{f_b}(x_i)$ で更新。すなわち残差を予測値で埋める。$\eta$は学習率。

最終的な予測値は下記のように残差の予測値を学習率をつけて足し合わせたもの。  
$$\hat{f}(x) = \sum_{b=1}^N \eta\hat{f_b}(x_i)$$

<br></br>
うまくいけばbiasとvarianceの両方を下げることができ、バギングよりも精度が高くなることが期待できる。  
弱学習器には決定木を採用することが多い。  
2023年3月時点で、このブースティングをベースとした勾配ブースティング決定木(GBDT)とニューラルネットが機械学習の主流。  
(画像認識、自然言語処理等ではニューラルネット、テーブルデータではGBDTの方が使われるぽい。)  


### AdaBoost(Adaptive Boosting)
基本的なブースティングのアルゴリズムの一つ。  
損失が大きい（上手く予測できなかった）データに重みをつけて次の学習で重点的に学習できるようにする手法。  
データに重みをつけるというのは、通常は各データの損失を表す関数ごとに重み$\omega_i$をつけるということ。  
これを全データ分足し合わせたものが弱学習器の損失関数になり、これが最小になるように学習する。  
$$L(x,y)=\frac{1}{m}\sum_{i=1}^m l(x_i,y_i)$$ 
この$l(x_i,y_i)$に対して損失の大きさに応じた重み$\omega_i$がつけられる。  
最初のイテレーションでは一律$\omega_i=\frac{1}{m}$がかけられ、  
次のイテレーションから各データの損失の大きさに応じて$\omega_i$は更新されていく。  
また、弱学習器自体の重み$\alpha$も損失に応じて計算され、  
AdaBoostでの最終的な予測値はこの重みと各弱学習器の予測結果を掛け合わせた合計となる。  


